In [2]:
#Import
import os
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA # this will make question and answering
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone


# I am going to import my PDF so i need to import DirectoryLoader
from langchain.document_loaders import PyPDFLoader,DirectoryLoader

# Corpus to Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


# we are using quant size model so we have to import ctransformer
from langchain.llms import CTransformers

c:\Users\arung\anaconda3\envs\chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
key=os.getenv("PINECONE_API_KEY")

In [4]:
# Dont use mine because i will delete it after completion of my project
PINECONE_API_KEY="1a0b20f7-7500-40fe-823a-dbdb343f593f"
PINECONE_API_ENV="gcp-starter"

# Load PDF

In [5]:
# load my pdf using Directory Loader
# We can load multiple pdf file also
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [6]:
extracted_data=load_pdf("Data/") # this pdf has 600 above pages that why its taking time

In [7]:
#print(extracted_data[300].page_content)# i have printed the 300th pages

# Load our and create text chunks
*   corpus to text_chunks

In [8]:
def text_split(extracted_data):
    text_spilitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spilitter.split_documents(extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)


In [9]:
print("Length of the chunks:",len(text_chunks))

Length of the chunks: 7020


# Embedding

In [10]:
# Now we are going to convert those 7020 chunks data to embedding and store vector DB 

# Download Embedding Model

# Previous Section we used OpenAI embedding model 

# Here we will use HuggingFaceEmbedding
def download_hugging_face_embedding(): # this func download the model huggingface itself
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # Available in HF portal
    return embeddings

embeddings=download_hugging_face_embedding() 
#It got downloaded

In [11]:
embeddings  # This embedding returns the 384 diementions

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [12]:
query=embeddings.embed_query("My name is karuppas")
print("Length",len(query))  # it return the 384 dienmentions
#Text embeddings

Length 384


In [13]:
import pinecone

pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name="medical-chat"

host="https://medical-chat-5y3awdy.svc.gcp-starter.pinecone.io"


#docs_search=Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [14]:
# from pinecone import Pinecone
# You can see vector counts in their page out index_name below we can see 7020
import pinecone
pc=pinecone.Pinecone(api_key=PINECONE_API_KEY)

from langchain_pinecone.vectorstores import Pinecone as pc_index

if index_name  in pc.list_indexes().names():
    vector_db=pc_index.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

